# Stream Objects Using Python

TODO: Summary

### Objectives

TODO

## Notebook Example

Implemented as notebook, not script. Code might look like this: 

In [3]:
from sys import argv
import pandas as pd
from my_functions import extract_temp_data, identify_season, create_histogram

SyntaxError: invalid syntax (1568491828.py, line 3)

In [ ]:
station_id = "USW00014837"

# Read data from file
station_data_filename = f"{station_id}.csv"
station_df = pd.read_csv(station_data_filename, low_memory=False)

In [ ]:
# Extract Min, Max Temperatures as dataframe
tempdf = extract_temp_data(station_df)

# Describe the dataset
print(f"{tempdf.describe()}\n")

# Label by season
tempdf['SEASON'] = tempdf.index.map(identify_season)

# Create histograms
# Saves as <STATION ID>-temp-dist.png, unless otherwise specified.
create_histogram(tempdf, station_id)

Suppose data was really big!!

In [ ]:
from pelicanfs.core import PelicanFileSystem

change data loading...

In [4]:
osdf_prefix = 'osdf:///aws-opendata/us-east-1/noaa-ghcn-pds'
station_URL = f"{osdf_prefix}/csv/by_station/{station_id}.csv"
pelfs = PelicanFileSystem(station_URL)

NameError: name 'PelicanFileSystem' is not defined

In [ ]:
station_data_filename = f"{station_id}.csv"
station_df = pd.read_csv(station_data_filename, low_memory=False)

Rest of analysis like normal